**Setup**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install librosa numpy pandas scipy scikit-learn soundfile

In [ ]:
!pip install synapseclient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successful

**Extract voice features**

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from scipy.signal import find_peaks, welch
from sklearn.decomposition import PCA
from google.colab import drive
import warnings

warnings.filterwarnings('ignore')

voice_data_path = '/content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/voice_data.csv'
voice_folder = '/content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/voice'
voice_data = pd.read_csv(voice_data_path, dtype={'recordId': str, 'healthCode': str})
recordid_to_healthcode = dict(zip(voice_data['recordId'], voice_data['healthCode']))

def calculate_jitter(y, sr, fmin, fmax):
    f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=fmin, fmax=fmax)
    f0 = f0[voiced_flag]
    if len(f0) < 2:
        return np.nan
    diff_f0 = np.diff(f0)
    jitter = np.mean(np.abs(diff_f0)) / np.mean(f0)
    return jitter

def calculate_shimmer(y, sr):
    amplitude_envelope = librosa.feature.rms(y=y).flatten()
    if len(amplitude_envelope) < 2:
        return np.nan
    diff_amplitude = np.diff(amplitude_envelope)
    shimmer = np.mean(np.abs(diff_amplitude)) / np.mean(amplitude_envelope)
    return shimmer

def calculate_hnr(y, sr):
    harmonic = librosa.effects.harmonic(y)
    percussive = librosa.effects.percussive(y)
    hnr = np.mean(harmonic / (percussive + 1e-10))
    return hnr

def get_cpp(y, sr):
    S = np.abs(librosa.stft(y))
    log_spectrum = np.log(S + 1e-10)  # Add a small value to avoid log(0)
    cepstrum = np.fft.ifft(log_spectrum, axis=0).real
    cepstrum = cepstrum.flatten()
    peaks, _ = find_peaks(cepstrum)
    return np.max(cepstrum[peaks]) if len(peaks) > 0 else 0

def calculate_psd(y, sr):
    f, Pxx = welch(y, sr, nperseg=1024)
    return np.mean(Pxx)

def calculate_h1_h2(y, sr):
    S = np.abs(librosa.stft(y))
    frequencies = librosa.fft_frequencies(sr=sr)
    idx = np.argsort(S.mean(axis=1))[-2:]  # Get indices of the top two harmonics
    h1 = S[idx[-1]].mean()
    h2 = S[idx[-2]].mean() if len(idx) > 1 else 0
    return h1 - h2

def calculate_gne(y, sr, max_freq=4500):
    S, phase = librosa.magphase(librosa.stft(y))
    harmonic = librosa.effects.harmonic(y)
    noise = y - harmonic
    gne = np.mean(librosa.feature.rms(y=harmonic) / (librosa.feature.rms(y=noise) + 1e-10))
    return gne

def calculate_hfno(y, sr, high_freq=6000):
    S = np.abs(librosa.stft(y))
    freqs = librosa.fft_frequencies(sr=sr)
    high_freq_mask = freqs > high_freq
    if not any(high_freq_mask):
        return np.nan
    hf_noise = np.mean(S[high_freq_mask])
    return hf_noise

def process_audio(audio_file):
    y, sr = librosa.load(audio_file, sr=None)
    fmin, fmax = librosa.note_to_hz('C1'), librosa.note_to_hz('C8')

    # Extracting time-varying features
    volume = librosa.feature.rms(y=y).flatten()
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
    pitch = pitches[pitches > 0]  # Filter out zero values

    zcr = librosa.feature.zero_crossing_rate(y).flatten()
    rms = librosa.feature.rms(y=y).flatten()
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr).flatten()
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr).flatten()
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr).flatten()

    # Extracting single value features
    jitter = calculate_jitter(y, sr, fmin, fmax)
    shimmer = calculate_shimmer(y, sr)
    hnr_d = calculate_hnr(y, sr)
    cpp = get_cpp(y, sr)
    psd = calculate_psd(y, sr)
    h1_h2 = calculate_h1_h2(y, sr)
    gne_max_4500hz = calculate_gne(y, sr, max_freq=4500)
    hfno_6000hz = calculate_hfno(y, sr, high_freq=6000)
    shim_db = 20 * np.log10(shimmer + 1e-10)

    # Calculate ABI
    abi = (5.0447730915 - (0.172 * cpp) - (0.193 * jitter) - (1.283 * gne_max_4500hz) -
           (0.396 * hfno_6000hz) + (0.01 * hnr_d) + (0.017 * h1_h2) +
           (1.473 * shim_db) - (0.088 * shimmer) - (68.295 * psd)) * 2.9257400394

    # Get recordId
    file_name = os.path.basename(audio_file)
    recordId = os.path.splitext(file_name)[0]

    # Feature dictionary
    data = {
        'recordId': recordId,
        'Volume_mean': np.mean(volume),
        'Volume_std': np.std(volume),
        'Pitch_mean': np.mean(pitch) if len(pitch) > 0 else np.nan,
        'Pitch_std': np.std(pitch) if len(pitch) > 0 else np.nan,
        'ZCR_mean': np.mean(zcr),
        'ZCR_std': np.std(zcr),
        'RMS_mean': np.mean(rms),
        'RMS_std': np.std(rms),
        'Spectral_Centroid_mean': np.mean(spectral_centroid),
        'Spectral_Centroid_std': np.std(spectral_centroid),
        'Spectral_Bandwidth_mean': np.mean(spectral_bandwidth),
        'Spectral_Bandwidth_std': np.std(spectral_bandwidth),
        'Spectral_Rolloff_mean': np.mean(spectral_rolloff),
        'Spectral_Rolloff_std': np.std(spectral_rolloff),
        'Jitter': jitter,
        'Shimmer': shimmer,
        'HNR_D': hnr_d,
        'CPP': cpp,
        'PSD': psd,
        'H1_H2': h1_h2,
        'GNE_max_4500Hz': gne_max_4500hz,
        'Hfno_6000Hz': hfno_6000hz,
        'Shim_dB': shim_db,
        'ABI': abi
    }

    # Extract MFCCs and compute PCA
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs = mfccs.T  # Transpose to have time steps as rows
    # Handle cases where MFCCs cannot be computed
    if mfccs.shape[0] < 2:
        data.update({f'PC{i}': np.nan for i in range(2, 12)})
    else:
        # Perform PCA
        pca = PCA(n_components=10)
        pca_result = pca.fit_transform(mfccs)
        for i in range(0, 10):
            data[f'PC{i+2}'] = pca_result[:, i].mean()

    return data

features_list = []

audio_files = [os.path.join(voice_folder, f) for f in os.listdir(voice_folder) if f.endswith('.m4a')]

for audio_file in audio_files:
    print(f"Processing {audio_file}")
    feature_data = process_audio(audio_file)
    recordId = feature_data['recordId']
    healthCode = recordid_to_healthcode.get(recordId, 'Unknown')
    feature_data['healthCode'] = healthCode
    features_list.append(feature_data)

df = pd.DataFrame(features_list)

desired_columns = [
    'recordId', 'healthCode',
    'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11',
    'Volume_mean', 'Volume_std', 'Pitch_mean', 'Pitch_std', 'ZCR_mean', 'ZCR_std',
    'RMS_mean', 'RMS_std', 'Spectral_Centroid_mean', 'Spectral_Centroid_std',
    'Spectral_Bandwidth_mean', 'Spectral_Bandwidth_std', 'Spectral_Rolloff_mean',
    'Spectral_Rolloff_std', 'Jitter', 'Shimmer', 'HNR_D', 'CPP', 'PSD', 'H1_H2',
    'GNE_max_4500Hz', 'Hfno_6000Hz', 'Shim_dB', 'ABI'
]

for col in desired_columns:
    if col not in df.columns:
        df[col] = np.nan

df = df[desired_columns]

output_csv_path = '/content/gdrive/MyDrive/path/pd_diagnosis/Zeno Paper/voice_features.csv'
df.to_csv(output_csv_path, index=False)

print("Feature extraction complete. Data saved to features.csv.")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Processing /content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/voice/f0608ed2-b581-41a3-b9f7-92110cc67f53.m4a
Processing /content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/voice/4432c292-c6d2-4f3a-8056-bdda7862cae4.m4a
Processing /content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/voice/91d63a7e-6189-42a7-be01-0035b49b96ce.m4a
Processing /content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/voice/382d9576-6460-40c7-8fee-f84a9bbfae29.m4a
Processing /content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/voice/ae14e9ab-1eae-45f2-838a-31ee0a37ba08.m4a
Processing /content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/voice/c8e9ce91-b3b4-44b5-a306-e93efe82ebbe.m4a
Processing /content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/voice/6e7d0cef-3896-4700-a043-05975abec4f4.m4a
Processing /content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/voice/aec12902-e93e-4077-b70a-9936d12a9841.m4a
Processing /content/g

OSError: Cannot save file into a non-existent directory: '/content/gdrive/MyDrive/path/pd_diagnosis/Zeno Paper'

**Add Demographic Information**

In [ ]:
import pandas as pd

voice_features_path = '/content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/voice_features.csv'
demographics_path = '/content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/demographics.csv'
output_csv_path = '/content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/voice+demographics.csv'

voice_features = pd.read_csv(voice_features_path, dtype={'healthCode': str})
demographics = pd.read_csv(demographics_path, dtype={'healthCode': str})

demographics_subset = demographics[
    ['healthCode', 'age', 'gender', 'packs-per-day', 'professional-diagnosis', 'smoked', 'years-smoking']
]

demographics_subset.rename(columns={
    'packs-per-day': 'packs',
    'professional-diagnosis': 'diagnosis'
}, inplace=True)

merged_df = pd.merge(voice_features, demographics_subset, on='healthCode', how='inner')
merged_df.to_csv(output_csv_path, index=False)

print("Merging complete. Data saved to merged_voice_demographics.csv.")

Merging complete. Data saved to merged_voice_demographics.csv.


**Extract Tapping and Gait Features**

In [ ]:
def get_tap_accel_for_healthCode(syn, table_id, healthCode):
    query = syn.tableQuery(f"SELECT * FROM {table_id} WHERE ( \"healthCode\" LIKE '{healthCode}' ) LIMIT 1")
    f = syn.downloadTableColumns(table=query, columns=["accel_tapping.json.items"])
    return f

def get_tap_for_healthCode(syn, table_id, healthCode):
    query = syn.tableQuery(f"SELECT * FROM {table_id} WHERE ( \"healthCode\" LIKE '{healthCode}' ) LIMIT 1")
    f = syn.downloadTableColumns(table=query, columns=["tapping_results.json.TappingSamples"])
    return f

def get_gait_accel_for_healthCode(syn, table_id, healthCode):
    query = syn.tableQuery(f"SELECT * FROM {table_id} WHERE ( \"healthCode\" LIKE '{healthCode}' ) LIMIT 1")
    f = syn.downloadTableColumns(table=query, columns=["deviceMotion_walking_outbound.json.items"])
    return f

In [ ]:
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

def calculate_magnitude(df):
    return np.sqrt(df['x']**2 + df['y']**2 + df['z']**2)

# Function for frequency feature extraction (for both tap and gait acceleration)
def calculate_frequency_features(df, column, sampling_rate):
    N = len(df[column])
    T = 1.0 / sampling_rate  # Sampling interval
    d = df[column].to_numpy()
    yf = fft(d)
    xf = fftfreq(N, T)[:N//2]
    power_spectrum = np.abs(yf[:N//2])

    dom_freq = xf[np.argmax(power_spectrum)]  # Dominant frequency
    mean_freq = np.sum(xf * power_spectrum) / np.sum(power_spectrum)  # Mean frequency

    spectral_centroid = np.sum(xf * power_spectrum) / np.sum(power_spectrum)
    spectral_spread = np.sqrt(np.sum(((xf - spectral_centroid) ** 2) * power_spectrum) / np.sum(power_spectrum))

    return dom_freq, mean_freq, spectral_centroid, spectral_spread

In [ ]:
import synapseclient
import pandas as pd
import json
from collections import OrderedDict
import numpy as np

syn = synapseclient.Synapse()
syn.login(authToken='[TOKEN]', silent=True)

voice_demographics_path = '/content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/voice+demographics.csv'
voice_demographics = pd.read_csv(voice_demographics_path, dtype={'healthCode': str})
healthCodes = voice_demographics['healthCode'].unique().tolist()

tapping_table_id = 'syn5511439'
gait_table_id = 'syn5511449'

features_list = []

for healthCode in healthCodes:
    print(f"Processing healthCode: {healthCode}")
    features = {'healthCode': healthCode}

    # Get tapping acceleration data (tap_accel)
    try:
        tap_accel_files = get_tap_accel_for_healthCode(syn, tapping_table_id, healthCode)
        for file_path in tap_accel_files.values():
            with open(file_path, 'r') as f:
                tap_accel_data = json.load(f)
                tap_accel_df = pd.DataFrame(tap_accel_data)
                features['tap_accel_rms_x'] = np.sqrt(np.mean(np.square(tap_accel_df['x'])))
                features['tap_accel_rms_y'] = np.sqrt(np.mean(np.square(tap_accel_df['y'])))
                features['tap_accel_rms_z'] = np.sqrt(np.mean(np.square(tap_accel_df['z'])))
                features['tap_accel_rms'] = np.sqrt(np.mean(tap_accel_df[['x', 'y', 'z']].pow(2).sum(axis=1)))

                features['tap_accel_std_x'] = tap_accel_df['x'].std()
                features['tap_accel_std_y'] = tap_accel_df['y'].std()
                features['tap_accel_std_z'] = tap_accel_df['z'].std()

                # Calculate frequency domain features (dominant frequency, mean frequency, etc.)
                sampling_rate = 50
                tap_accel_magnitude = calculate_magnitude(tap_accel_df)
                tap_accel_magnitude_df = pd.DataFrame({'magnitude': tap_accel_magnitude})
                print(tap_accel_magnitude_df['magnitude'])
                dom_freq, mean_freq, spectral_centroid, spectral_spread = calculate_frequency_features(tap_accel_magnitude_df, 'magnitude', sampling_rate)
                features['tap_accel_dom_freq'] = dom_freq
                features['tap_accel_mean_freq'] = mean_freq
                features['tap_accel_spectral_centroid'] = spectral_centroid
                features['tap_accel_spectral_spread'] = spectral_spread
    except Exception as e:
        print(f"No tap acceleration data for healthCode {healthCode}: {e}")

    # Get tapping results data
    try:
        tap_results_files = get_tap_for_healthCode(syn, tapping_table_id, healthCode)
        for file_path in tap_results_files.values():
            with open(file_path, 'r') as f:
                tap_results_data = json.load(f)
                tap_results_df = pd.DataFrame(tap_results_data)
                tap_results_df = tap_results_df[tap_results_df['TappedButtonId'] != 'TappedButtonNone']

                # Extract tapping features
                features['number_of_taps'] = len(tap_results_df)
                features['left_taps'] = (tap_results_df['TappedButtonId'] == 'TappedButtonLeft').sum()
                features['right_taps'] = (tap_results_df['TappedButtonId'] == 'TappedButtonRight').sum()
                features['total_taps'] = features['left_taps'] + features['right_taps']
                features['repeated_taps'] = (tap_results_df['TappedButtonId'] == tap_results_df['TappedButtonId'].shift()).sum()

                # Calculate inter-tap interval (ITI) consistency (std of intervals)
                if 'TapTimeStamp' in tap_results_df.columns:
                    tap_results_df = tap_results_df.sort_values(by='TapTimeStamp')
                    inter_tap_intervals = tap_results_df['TapTimeStamp'].diff().dropna()
                    features['tap_iti_consistency'] = inter_tap_intervals.std()
                else:
                    features['tap_iti_consistency'] = np.nan
    except Exception as e:
        print(f"No tapping results data for healthCode {healthCode}: {e}")

    # Get gait acceleration data (gait_accel)
    try:
        gait_accel_files = get_gait_accel_for_healthCode(syn, gait_table_id, healthCode)
        for file_path in gait_accel_files.values():
            with open(file_path, 'r') as f:
                gait_accel_data = json.load(f)
                user_accel_data = [entry['userAcceleration'] for entry in gait_accel_data]
                gait_accel_df = pd.DataFrame(user_accel_data)

                # Extract features for gait acceleration (RMS, STD, etc.)
                features['gait_accel_rms_x'] = np.sqrt(np.mean(np.square(gait_accel_df['x'])))
                features['gait_accel_rms_y'] = np.sqrt(np.mean(np.square(gait_accel_df['y'])))
                features['gait_accel_rms_z'] = np.sqrt(np.mean(np.square(gait_accel_df['z'])))
                features['gait_accel_rms'] = np.sqrt(np.mean(gait_accel_df[['x', 'y', 'z']].pow(2).sum(axis=1)))

                features['gait_accel_std_x'] = gait_accel_df['x'].std()
                features['gait_accel_std_y'] = gait_accel_df['y'].std()
                features['gait_accel_std_z'] = gait_accel_df['z'].std()

                # Calculate frequency domain features (dominant frequency, mean frequency, etc.)
                sampling_rate = 50
                gait_accel_magnitude = calculate_magnitude(gait_accel_df)
                gait_accel_magnitude_df = pd.DataFrame({'magnitude': gait_accel_magnitude})
                dom_freq, mean_freq, spectral_centroid, spectral_spread = calculate_frequency_features(gait_accel_magnitude_df, 'magnitude', sampling_rate)
                features['gait_accel_dom_freq'] = dom_freq
                features['gait_accel_mean_freq'] = mean_freq
                features['gait_accel_spectral_centroid'] = spectral_centroid
                features['gait_accel_spectral_spread'] = spectral_spread
    except Exception as e:
        print(f"No gait acceleration data for healthCode {healthCode}: {e}")

    features_list.append(features)

features_df = pd.DataFrame(features_list)
print(features_df.head())
features_df.to_csv('extracted_features.csv', index=False)

Processing healthCode: 261dd050-4f8e-4582-889e-ff7081706d9e


INFO:synapseclient_default:Downloading 0 files, 1 cached locally


0       1.007126
1       1.010013
2       1.007881
3       1.011374
4       1.009907
          ...   
2034    1.007303
2035    1.009854
2036    1.011470
2037    1.000279
2038    1.006129
Name: magnitude, Length: 2039, dtype: float64


INFO:synapseclient_default:Downloading 0 files, 1 cached locally


INFO:synapseclient_default:Downloading 0 files, 1 cached locally


...

INFO:synapseclient_default:Downloaded syn5511439 to /tmp/tmpqg3jja2v/table_file_download.zip

Downloaded syn5511449 to /root/.synapseCache/423/148006423/SYNAPSE_TABLE_QUERY_148006423.csv


INFO:synapseclient_default:Downloaded syn5511449 to /root/.synapseCache/423/148006423/SYNAPSE_TABLE_QUERY_148006423.csv


INFO:synapseclient_default:Downloading 0 files, 0 cached locally


                             healthCode  tap_accel_rms_x  tap_accel_rms_y  \
0  261dd050-4f8e-4582-889e-ff7081706d9e         0.042560         0.033806   
1  83a357f0-af9c-4da1-9dad-42df2475526d         0.126463         0.037296   
2  7bc86881-7075-467a-850e-53e021b60f9f         0.038730         0.022164   
3  d49fcd5c-3952-49ba-bc03-261b83a7ab6e         0.142102         0.791075   
4  1ee9c522-d68a-4d32-8113-4c5246986978         0.013560         0.010034   

   tap_accel_rms_z  tap_accel_rms  tap_accel_std_x  tap_accel_std_y  \
0         1.009058       1.010521         0.042071         0.033815   
1         0.993163       1.001876         0.033574         0.036183   
2         1.007938       1.008926         0.019887         0.014309   
3         0.587237       0.995409         0.022827         0.019403   
4         1.025605       1.025744         0.002039         0.003785   

   tap_accel_std_z  tap_accel_dom_freq  tap_accel_mean_freq  ...  \
0         0.059470                 0.0    

**Combine tapping and gait with voice and demographics**

In [ ]:
voice_demographics = voice_demographics.merge(features_df, on='healthCode', how='inner')
voice_demographics.head()

,recordId,healthCode,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,gait_accel_rms_y,gait_accel_rms_z,gait_accel_rms,gait_accel_std_x,gait_accel_std_y,gait_accel_std_z,gait_accel_dom_freq,gait_accel_mean_freq,gait_accel_spectral_centroid,gait_accel_spectral_spread
0,f0608ed2-b581-41a3-b9f7-92110cc67f53,261dd050-4f8e-4582-889e-ff7081706d9e,0.000009,-0.000026,0.000008,0.000003,-9.855735e-07,3.308711e-06,3.766298e-06,-3.137115e-06,...,0.251464,0.217071,0.429346,0.255863,0.248148,0.216806,0.0,6.599959,6.599959,6.530913
1,4432c292-c6d2-4f3a-8056-bdda7862cae4,83a357f0-af9c-4da1-9dad-42df2475526d,0.000018,-0.000012,-0.000002,-0.000006,1.504711e-05,4.930441e-06,1.345949e-05,-8.820109e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,91d63a7e-6189-42a7-be01-0035b49b96ce,7bc86881-7075-467a-850e-53e021b60f9f,0.000003,0.000007,-0.000002,-0.000002,3.245800e-06,5.292065e-07,-4.939261e-07,3.069398e-06,...,0.312996,0.456423,0.608307,0.251782,0.299914,0.409046,0.0,5.279880,5.279880,5.455245
3,382d9576-6460-40c7-8fee-f84a9bbfae29,d49fcd5c-3952-49ba-bc03-261b83a7ab6e,-0.000017,-0.000004,-0.000005,0.000006,-1.215769e-06,-1.885324e-06,-2.836796e-06,6.695542e-07,...,0.429279,0.349057,0.644937,0.330372,0.427223,0.348915,0.0,5.526321,5.526321,5.591330
4,ae14e9ab-1eae-45f2-838a-31ee0a37ba08,1ee9c522-d68a-4d32-8113-4c5246986978,0.000021,-0.000003,-0.000004,-0.000003,2.759942e-06,1.353602e-06,1.089914e-06,1.190994e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Ensure all values are numerical and not N/A**

In [ ]:
voice_demographics = voice_demographics.dropna(subset=['diagnosis'])
voice_demographics['diagnosis'] = voice_demographics['diagnosis'].map({True: 1, False: 0})

if 'gender' in voice_demographics.columns:
    voice_demographics['gender'] = voice_demographics['gender'].map({'Male': 1, 'Female': 0})

gait_columns = ['gait_accel_rms_x', 'gait_accel_rms_y', 'gait_accel_rms_z', 'gait_accel_rms',
                'gait_accel_std_x', 'gait_accel_std_y', 'gait_accel_std_z',
                'gait_accel_mean_freq', 'gait_accel_spectral_centroid', 'gait_accel_spectral_spread']
voice_demographics = voice_demographics.dropna(subset=gait_columns)

if 'packs' in voice_demographics.columns:
    smokers = voice_demographics[voice_demographics['smoked'] == True]
    smokers_packs_mean = smokers['packs'].mean()
    smokers_years_mean = smokers['years-smoking'].mean()
    voice_demographics['packs'] = voice_demographics['packs'].fillna(
        voice_demographics.apply(
            lambda row: smokers_packs_mean if row['packs'] is None and row['smoked'] == 'True'
            else (0 if row['smoked'] == 'False' else voice_demographics['packs'].mean()), axis=1)
    )
    voice_demographics['years-smoking'] = voice_demographics['years-smoking'].fillna(
        voice_demographics.apply(
            lambda row: smokers_years_mean if row['years-smoking'] is None and row['smoked'] == 'True'
            else (0 if row['smoked'] == 'False' else voice_demographics['years-smoking'].mean()), axis=1)
    )

voice_demographics['smoked'] = voice_demographics['smoked'].map({True: 1, False: 0})
voice_demographics['smoked'] = voice_demographics['smoked'].fillna(voice_demographics['smoked'].mean())

[WARNING] <ipython-input-265-f5c915f3af1e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voice_demographics['packs'] = voice_demographics['packs'].fillna(

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voice_demographics['packs'] = voice_demographics['packs'].fillna(

[WARNING] <ipython-input-265-f5c915f3af1e>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

**Normalize Features**

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select the numerical columns (excluding binary columns such as 'diagnosis' and 'gender')
columns_to_exclude = ['diagnosis', 'sex', 'smoked']
numerical_cols = voice_demographics.select_dtypes(include=['float64', 'int64']).columns
numerical_cols = [col for col in numerical_cols if col not in columns_to_exclude]

# Initialize the StandardScaler
scaler = StandardScaler()

# Apply the StandardScaler to the numerical columns
voice_demographics[numerical_cols] = scaler.fit_transform(voice_demographics[numerical_cols])

voice_demographics.head()

,recordId,healthCode,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,gait_accel_rms_y,gait_accel_rms_z,gait_accel_rms,gait_accel_std_x,gait_accel_std_y,gait_accel_std_z,gait_accel_dom_freq,gait_accel_mean_freq,gait_accel_spectral_centroid,gait_accel_spectral_spread
0,f0608ed2-b581-41a3-b9f7-92110cc67f53,261dd050-4f8e-4582-889e-ff7081706d9e,0.599165,-3.553556,1.712746,0.646849,-0.405648,1.209048,1.627549,-1.413236,...,-0.012576,-0.263329,-0.043021,0.224374,0.033868,-0.163448,0.0,1.513348,1.513348,1.403931
2,91d63a7e-6189-42a7-be01-0035b49b96ce,7bc86881-7075-467a-850e-53e021b60f9f,0.193467,1.082631,-0.481740,-0.574674,1.068973,0.183910,-0.224288,1.233481,...,0.408375,1.507706,0.729936,0.191261,0.445651,1.417896,0.0,0.257841,0.257841,-0.082680
3,382d9576-6460-40c7-8fee-f84a9bbfae29,d49fcd5c-3952-49ba-bc03-261b83a7ab6e,-1.100454,-0.451838,-0.944197,1.545152,-0.485870,-0.706618,-1.242688,0.210088,...,1.203881,0.713274,0.888146,0.828928,1.458341,0.923263,0.0,0.492227,0.492227,0.105395
5,c8e9ce91-b3b4-44b5-a306-e93efe82ebbe,34e2480b-3454-458b-86b0-47e176741b60,1.562682,-0.294987,-1.094885,0.113474,-0.431032,0.353068,0.803205,0.586544,...,-0.680230,-0.785442,-0.744389,-0.567200,-0.717305,-0.757088,0.0,2.414406,2.414406,2.505850
6,6e7d0cef-3896-4700-a043-05975abec4f4,83c7de09-2ed3-409b-abdb-fc75750d10fe,-0.398626,0.413502,0.550139,-0.765826,1.241105,0.014752,0.542498,0.406003,...,-0.531793,-0.534428,-0.564980,-0.404805,-0.561383,-0.607498,0.0,-0.077595,-0.077595,0.086567


In [ ]:
voice_demographics.to_csv('/content/gdrive/MyDrive/pd_diagnosis/Zeno Paper/normalized_df.csv')